## 1. 데이터 불러오기

'1_훼손 영역에 대한 검출 가능성.ipynb'에서 생성한 데이터를 사용.

In [1]:
import os

DATA_STR_PATH = "./data/"

# --------------------------------

if not os.path.exists(DATA_STR_PATH): raise Exception("No data.")

file_basename_list = []

with open(f"{DATA_STR_PATH}.datalist", "r") as f:
    file_basename_list = f.readlines()

def load():
    for file_basename in file_basename_list:
        file_name, file_extension = os.path.splitext(file_basename)

        raw  = cv2.imread(f"{DATA_STR_PATH}{file_basename}", cv2.IMREAD_COLOR)
        mask = cv2.imread(f"{DATA_STR_PATH}{file_name}_mask{file_extension}", cv2.IMREAD_GRAYSCALE)

        yield raw, mask

## 2. 

In [2]:
import cv2
import numpy as np

MIN_CONT_AREA = 20

# --------------------------------

for raw, mask in load():
    contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

    labled_mask = np.zeros(mask.shape, dtype=np.uint8)

    for cont in contours:
        if cv2.contourArea(cont) >= MIN_CONT_AREA:
            cv2.drawContours(labled_mask, [cont], -1, 1, -1)

        else:
            cv2.drawContours(labled_mask, [cont], -1, 2, -1)


(203, 301, 3) (203, 301)
